# Projeto


## Binário de rotatividade de clientes

Uma agência de marketing tem muitos clientes que usam seus serviços para produzir anúncios para os sites de clientes / clientes. Eles perceberam que há uma grande rotatividade de clientes. Eles basicamente atribuem gerentes de conta de forma aleatória agora, mas querem que você crie um modelo de aprendizado de máquina que ajude a prever quais clientes abandonarão (pararão de comprar seu serviço) para que possam atribuir corretamente os clientes com maior risco a dispensar um gerente de conta. Felizmente, eles têm alguns dados históricos, você pode ajudá-los? Crie um algoritmo de classificação que ajudará a classificar se um cliente abandonou ou não. Em seguida, a empresa pode testar isso em relação aos dados recebidos para futuros clientes, a fim de prever quais clientes irão se desligar e atribuir a eles um gerente de conta.

Os dados são salvos como customer_churn.csv. Aqui estão os campos e suas definições:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Depois de criar o modelo e avaliá-lo, teste o modelo em alguns novos dados (você pode pensar nisso quase como um conjunto de validação) que seu cliente forneceu, salvo em new_customers.csv. O cliente deseja saber quais clientes têm maior probabilidade de se desligar com base nesses dados (eles ainda não têm o rótulo).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



### Confira os dados

In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Formato para MLlib

Usaremos as colunas numéricas. Incluiremos o gerente de contas porque é muito fácil, mas lembre-se de que provavelmente não será um sinal porque a agência mencionou que foi atribuído aleatoriamente!

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
final_data = output.select('features','churn')

### Test Train Split

In [11]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Treinar o modelo

In [12]:
from pyspark.ml.classification import LogisticRegression

In [13]:
lr_churn = LogisticRegression(labelCol='churn')

In [14]:
fitted_churn_model = lr_churn.fit(train_churn)

In [15]:
training_sum = fitted_churn_model.summary

In [16]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               660|                660|
|   mean|0.1621212121212121|0.12878787878787878|
| stddev|0.3688414999063377|0.33521906479293934|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



### Avaliando os resultados

Vamos avaliar os resultados no conjunto de dados que recebemos (usando os dados de teste)

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [18]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [19]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.63863097908381...|[0.99042170275359...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.39628601471155...|[0.80159387368445...|       0.0|
|[28.0,11245.38,0....|    0|[3.78866997367730...|[0.97787492026371...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.14462067214799...|[0.98439783868026...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.75362193950544...|[0.99145326048336...|       0.0|
|[29.0,13240.01,1....|    0|[6.90018999117996...|[0.99899342024680...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.42933656493430...|[0.96860890181628...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.01475561047634...|[0.99756350580391...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.34838937470954...|[0.98723737298951...|       0.0|
|[30.0,10744.14,

### Usando AUC

In [20]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [21]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [22]:
auc

0.7547515051351671

[Common question - what is a good AUC value?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Preveja novos dados não rotulados

Ainda precisamos avaliar o arquivo new_customers.csv!

In [23]:
final_lr_model = lr_churn.fit(final_data)

In [24]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [25]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [26]:
test_new_customers = assembler.transform(new_customers)

In [27]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [28]:
final_results = final_lr_model.transform(test_new_customers)

In [30]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Agora sabemos que devemos atribuir gerentes de contas a Cannon-Benson, Barron-Robertson, Sexton-GOlden e Parks-Robbins!